기한: 3월 22일 (금) 18:30

과제: 핸즈온 7장 앙상블 학습과 랜덤 포레스트

p. 245-271 필사하여 깃허브에 업로드한 뒤, 깃허브 링크를 댓글로 달아주세요.


In [2]:
import warnings 
warnings.filterwarnings('ignore')

# import package

import numpy as np 
import os

In [3]:
#5장에서 소개한 moons dataset 불러오기
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X,y = make_moons(n_samples=100, noise=0.15)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Chapter7. 앙상블 학습과 랜덤 포레스트

- 앙상블 학습 : 
- 결정 트리의 앙상블 : 랜덤 포레스트

## 7.1 투표 기반 분류기

정확도가 80%인 분류기 여러 개를 훈련시켰다고 가정해보자. 더 좋은 분류기를 만드는 매우 간단한 방법은 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것이다. 
- 직접 투표 분류기 (hard voting) : 다수결 투표로 정해지는 분류기
- 각 분류기가 약한 학습기 (weak learner, 랜덤 추측보다 조금 더 높은 성능을 내는 분류기) 일지라도 충분하게 많고 다양하다면 앙상블은 강한 학습기 (strong learner, 높은 정확도를 내는 분류기)가 될 수 있다.
- 이러한 다수결 투표 분류기가 앙상블에 포함된 개별 분류기 중 가장 뛰어난 것보다도 정확도가 높은 경우가 많다.

[예시]
동전을 던졌을 때 앞면이 51%, 뒷면이 49%로 나오는 조금 균형이 맞지 않는 동전이 있다고 가정해보자. 이 동전을 1000번 던진다면 앞면은 대략 510번, 뒷면은 대랴 490번이 나올 것이므로 다수는 앞면이 된다. 이를 수학적으로 계산해본다면, 동전을 1000번 던진 후 앞면이 다수가 될 확률은 75%에 가깝다. 만약 10000번 던진다면 확률은 97%까지 올라간다. 이를 "큰 수의 법칙" 이라고 말한다.

------------------------

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.ensemble import VotingClassifier

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], 
    voting = 'hard'
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

각 분류기별 테스트셋 정확도 확인

In [5]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8


RandomForestClassifier 0.95
SVC 1.0
VotingClassifier 0.95


###### 어라리 투표 기반 분류기가 다른 분류기보다 성능이 높아야되는데..

----------------------
- 간접 투표 (soft voting) : 개별 분류기의 예측을 평균내어 확률이 가장 높은 클래스를 예측한다.
- 이 방식은 확률이 높은 투표에 더 많은 비중을 두기 때문에 직접 투표 방식(hard voting)보다 성능이 좋다.
- 간접 투표에서 사용하는 분류기가 모두 클래스의 확률을 계산할 수 있어야한다. (predict_proba() 메서드 사용이 가능해야한다.)
- SVC는 기본값에서는 클래스 확률을 제공하지 않으므로 probabiltiy 매개변수를 True로 지정해야 한다. 다만 클래스 확률을 추정하기 위해 교차 검증을 사용하므로 훈련 속도는 느려진다.

In [6]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], 
    voting = 'soft'
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))],
                 voting='soft')

In [7]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8


RandomForestClassifier 0.9
SVC 1.0
VotingClassifier 0.95


---------------------
## 7.2 배깅과 페이스팅

- 배깅(bagging) : 훈련세트에서 중복을 허용하여 샘플링하는 방식
(bootstrap aggregating의 줄임말)
- 페이스팅(pasting) : 훈련세트에서 중복을 허용하지 않고 샘플링하는 방식
- 통계학에서는 중복을 허용한 리샘플링을 부트스트래핑이라고 한다. 

모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측을 만든다. 수집 함수는 {분류 : 통계적 최빈값, 회귀 : 평균}으로 계산한다.



개별 예측기는 원본 훈련 세트로 훈련시킨 것보다 훨씬 크게 편향되어 있지만, 수집 함수를 통과하면 편향과 분산이 모두 감소한다. 일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련 시킬때와 비교하여 편향은 비슷하지만 분산은 줄어든다.

-------------

### 7.2.1 사이킷런의 배깅과 페이스팅

사이킷런의 BaggingClassifier (분류), BaggingRegressor (회귀)는 배깅과 페이스팅 기능을 제공한다. 다음은 결정 트리 분류기 500개의 앙상블을 훈련시키는 코드이다. 각 분류기는 훈련 세트에서 중복을 허용하여 무작위로 선택된 100개의 샘플로 훈련된다.

- 페이스팅 : bootstrap = False로 지정

In [8]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500, 
    max_samples = 50, bootstrap = True, n_jobs = -1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

전반적으로 배깅이 더 나은 모델을 만들기 때문에 일반적으로 더 선호된다. 그러나 시간과 CPU 파워에 여유가 있다면 교차 검증으로 배깅과 페이스팅을모두 평가해서 더 나은 쪽을 선택하는 것이 좋다.

-------
### 7.2.2 oob 평가

배깅을 사용하면 어떤 샘플은 여러 번 샘플링 되고 어떤 샘플은 전혀 선택되지 않을 수 있다. BaggingClassifier는 기본값으로 중복을 허용하여 훈련 세트의 크기만큼인 m개 샘플을 선택한다. 이는 평균적으로 각 예측기에 훈련샘플의 63% 정도만 샘플링 된다는 것을 의미한다. 선택되지 않은 훈련 샘플의 나머지 37%를 out-of-sample, oob라고 부른다. 예측기마다 남겨진 37%는 모두 다르다.

예측기가 훈련되는 동안에는 oob 샘플은 사용하지 않으므로 별도의 검증 세트를 사용하지 않고 oob 샘플을 사용해 평가할 수 있다. 앙상블에서의 평가는 각 예측기의 oob 평가를 평균하여 얻는다.

BaggingClassifier를 만들 때 oob_score = True로 지정하면 훈련이 끝난 후 자동으로 oob 평가를 수행한다. 평가 점수 결과는 oob_score_ 변수에 저장되어 있다.

In [9]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap = True, n_jobs = -1, oob_score = True
)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.925

In [10]:
from sklearn.metrics import accuracy_score

y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

- oob_decision_function : oob 샘플에 대한 결정 함수의 값을 확인할 수 있다.

In [11]:
bag_clf.oob_decision_function_

array([[0.93296089, 0.06703911],
       [0.97826087, 0.02173913],
       [0.74404762, 0.25595238],
       [0.90425532, 0.09574468],
       [1.        , 0.        ],
       [0.78609626, 0.21390374],
       [0.        , 1.        ],
       [0.02840909, 0.97159091],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.93820225, 0.06179775],
       [0.        , 1.        ],
       [0.09189189, 0.90810811],
       [1.        , 0.        ],
       [0.86010363, 0.13989637],
       [0.94358974, 0.05641026],
       [0.93922652, 0.06077348],
       [0.48      , 0.52      ],
       [0.9408284 , 0.0591716 ],
       [0.95108696, 0.04891304],
       [0.88709677, 0.11290323],
       [0.        , 1.        ],
       [0.99438202, 0.00561798],
       [0.        , 1.        ],
       [0.77647059, 0.22352941],
       [0.01724138, 0.98275862],
       [0.6142132 , 0.3857868 ],
       [0.        , 1.        ],
       [0.00588235, 0.99411765],
       [0.96      , 0.04      ],
       [0.

## 7.3 랜덤 패치와 랜덤 서브스페이스

BaggingClassifier는 특성에 대한 샘플링도 지원한다.
- max_features
- bootstrap_features

1. 랜덤 패치 방식
2. 랜덤 서브스페이스 방식

특성 샘플링은 더 다양한 예측기를 만들어 편향을 늘리는 대신 분산을 낮춘다

## 7.4 랜덤 포레스트

일반적으로 배깅 방법을 적용하는 트리 형태의 앙상블

In [12]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입한다. 이는 결국 트리를 더욱 다양하게 만들고 편향을 보는 대신 분산을 낮추어 더 훌륭한 모델을 만든다.


아래 코드는 BaggingClassifier를 사용해 RandomForestClassifier와 유사하게 만든 것.

In [13]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

### 7.4.1 엑스트라 트리

- 익스트림 랜덤 트리 (엑스트라 트리) : 극단적으로 무작위한 트리의 랜덤 포레스트. 최적의 임곗값을 찾는 대신 후보 특성을 사용해 무작위로 분할한 다음 그 중 최상의 분할을 선택한다.
- 속도가 매우 빠르다.
- ExtraTreesClassifier, ExtraTreesRegressor 사용

### 7.4.2 특성 중요도

- 랜덤 포레스트는 특성의 상대적 중요도를 측정하기 쉽다.
- 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도를 측정한다.
- feature_importances_ 변수에 저장되어있다.

In [14]:
from sklearn.datasets import load_iris
iris=load_iris()

rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.09440092304138675
sepal width (cm) 0.02277269921378
petal length (cm) 0.43156891273648085
petal width (cm) 0.4512574650083525


## 7.5 부스팅

- 부스팅 : 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법
- 앞의 모델을 보완해나가면서 일련의 예측기를 학습시킨다.
- AdaBoost
- GradientBoost

### 7.5.1 에이다부스트 (AdaBoost)

- AdaBoost : 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이는 방식으로 훈련한다.

In [15]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200, algorithm="SAMME.R", learning_rate=0.5
)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### 7.5.2 그래디언트 부스팅 (GradientBoosting)

- GradientBoosting : 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습시킨다.
- GradientTreeBoosting 또는 Gradient Boosted Regression Tree (GBRT)

In [16]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth = 2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2)

첫 번째 예측기에서 생긴 잔여 오차에 두 번째 DecisionTreeRegressor를 훈련시킨다.

In [17]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth = 2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

두 번째 예측기가 만든 잔여 오차에 세 번째 회귀 모델을 훈련시킨다.

In [18]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth = 2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

이제 세 개의 트리를 포함하는 앙상블 모델이 생겼다. 새로운 샘플에 대한 예측을 만들려면 모든 트리의 예측을 더하면 된다.

In [19]:
# y_pred = sum(tree.predcit(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [ ]:
# 테스트셋 생성
X_new = np.array([[0.8]])
#y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

GradientBoostingRegressor를 사용하면 GBRT 앙상블을 간단하게 훈련시킬 수 있다.

In [20]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 3, learning_rate = 1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

- 축소 (shrinkage) : 학습률을 낮게 설정하면 앙상블을 학습시키기 위해 많은 트리가 필요하지만 일반적으로 성능은 좋아진다.

- 조기 종료 기법 : 최적의 트리수를 찾기 위해 사용가능.
- staged_predict() 메서드 사용시 간단하게 구현 가능.

In [21]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X,y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth = 2, n_estimators = bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=35)

실제로 훈련을 중지하는 방법으로 조기 종료를 구현할 수도 있다.
- warm_start = True로 설정시 사이킷런 fit() 메서드가 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있도록 한다.

In [24]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up=0

for n_estimators in range(1,120):
  gbrt.n_estimators=n_estimators
  gbrt.fit(X_train, y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val, y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up = 0
  else :
    error_going_up += 1
    if error_going_up == 5:
      break # 조기종료

- 확률적 그래디언트 부스팅 (stochastic gradient boosting)
- 대표적으로 XGBoost 라이브러리로 구현할 수 있다.

In [25]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [26]:
xgb_reg.fit(X_train, y_train,
             eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.40122
[1]	validation_0-rmse:0.34565
[2]	validation_0-rmse:0.30094
[3]	validation_0-rmse:0.27674
[4]	validation_0-rmse:0.27802
[5]	validation_0-rmse:0.27862


## 7.6 스태킹

- 스태킹(stacked generalization) : 앙상블에 속한 모든 예측기의 예측을 취합하는 함수를 사용하는 대신 취합하는 모델을 훈련시킬 수 없을까요? 라는 아이디어

- 마지막 예측기(블렌더 또는 메타학습기)가 예측기마다 다른 예측을 입력받아 최종 예측한다.

- 블렌더를 학습시키는 방법 : 홀드 아웃 (hold-out) 세트 이용

1. 훈련 세트를 두 개의 서브셋으로 나눈다.
2. 첫 번째 서브셋은 첫 번째 레이어의 예측을 훈련하기 위해 사용된다
3. 첫 번째 레이어의 예측기를 사용해 두 번째 (홀드 아웃) 세트에 대한 예측을 만든다.
4. 홀드 아웃 세트의 각 샘플에 대해 세 개의 예측값이 있다.
5. 타깃값은 그대로 쓰고 앞에서 예측한 값을 입력 특성으로 사용하는 새로운 훈련 세트를 만든다.
6. 블렌더가 새 훈련 세트로 훈련된다. 즉 첫 번째 레이어의 예측을 가지고 타깃값을 예측하도록 학습된다.